In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os,glob
import time
!pip3 install requests
import requests
import http

In [ ]:
# create /talks/ if do not exist
if not os.path.exists('talks'):
    os.makedirs('talks')

# clear the pwd when needed
def delCSV ():
  filenames = glob.glob(os.getcwd() + "/*.csv")
  for filename in filenames:
    os.remove(filename)

delCSV ()
os.chdir('talks')
delCSV ()
os.chdir('/content/')

In [ ]:
sleep = 3
def requestTed(path):
  r = requests.get(path)
  # sleep to wait
  time.sleep(sleep)
  return r

In [ ]:
# EXTRACT TED Talk Names
def enlist_talk_names(TargetPath, dict_, lang, page):
  main = TargetPath + lang
  for i in range(1, page + 1):
    path = main + '&page=' + str(i)
    r = requestTed(path)
    print (i, path)

    soup = BeautifulSoup(r.text)
    talks= soup.find_all("a",class_='')
    for i in talks:
      href = i.attrs['href'][:-(10+len(lang))]
      if href.find('/talks/')==0 and dict_.get(href)!= 1:
        print(href)
        dict_[href]=1
      
  return dict_

In [ ]:
TargetLang = ['zh', 'en']
TargetPath = 'https://www.ted.com/talks?language='
# noise = [' (Applause) ', ' （掌聲） ', ' (Laughter) ', '（笑聲） ', ' (Guitar) ', ' （吉他）']
noise = []
all_talk_names = enlist_talk_names(TargetPath, {}, TargetLang[0], page = 6)
print()
print('Total numbere of talks:', (len(all_talk_names)))

1 https://www.ted.com/talks?language=zh&page=1
/talks/kristen_bell_giant_ant_why_act_now
/talks/brent_loken_can_we_create_the_perfect_farm
/talks/matt_walker_how_sleep_can_improve_your_immunity
/talks/matt_walker_how_sleep_affects_your_emotions
/talks/matt_walker_how_caffeine_and_alcohol_affect_your_sleep
/talks/shekinah_elmore_the_courage_to_live_with_radical_uncertainty
/talks/aparna_pallavi_what_foods_did_your_ancestors_love
/talks/elizabeth_cox_what_is_a_coronavirus
/talks/anees_bahji_what_is_schizophrenia
/talks/jim_hudspeth_the_beautiful_mysterious_science_of_how_you_hear
/talks/iseult_gillespie_the_tale_of_the_doctor_who_defied_death
/talks/kristen_wenz_what_if_a_single_human_right_could_change_the_world
/talks/bill_schutt_how_do_blood_transfusions_work
/talks/a_j_jacobs_the_evolution_of_the_coffee_cup_lid
/talks/jen_gunter_why_can_t_we_talk_about_periods
/talks/rod_phillips_a_brief_history_of_alcohol
/talks/elizabeth_waters_the_left_brain_vs_right_brain_myth
/talks/wendy_suzuki

In [ ]:
def extract_talk(name, lang):
  main = 'https://www.ted.com' + name + '/transcript?/transcript'
  parallel = {}
  for i in lang:
    path = main + '&language=' + i
    print(path)
    r = requestTed(path)
    soup = BeautifulSoup(r.text)
    # if failed, try again
    while len(soup.findAll('p')) == 0:
      r = requestTed(path)
      soup = BeautifulSoup(r.text)

    text_talk = []
    for x in soup.findAll('p'):
      x = x.text.replace('\n',' ').replace('\t','')
      if x not in noise:
        text_talk.append(x)
    text_talk = text_talk[:len(text_talk)-2]
    parallel[i] = text_talk

  p_length = [len(parallel[x]) for x in TargetLang]
  if p_length[0] == p_length[1]:
    if p_length[0] != 0:
      for x in TargetLang:
        print(parallel[x][0])
      df = pd.DataFrame(parallel)
      df.to_csv(os.getcwd() + name + '.csv', sep='\t', encoding='utf-8')
      return 1, p_length[0]
    else:
      print('Cannot retreive subtitle for both')
      print(p_length[0], p_length[1])
  else:
    print('Different length between both languages')
    print(p_length[0], p_length[1])
  return 0, 0

In [ ]:
count = 0
corpus_length = 0
start = time.time()
for i in all_talk_names:
  c, l = extract_talk(i, TargetLang)
  count += c
  corpus_length += l
end = time.time()

https://www.ted.com/talks/kristen_bell_giant_ant_why_act_now/transcript?/transcript&language=zh
https://www.ted.com/talks/kristen_bell_giant_ant_why_act_now/transcript?/transcript&language=en
 點解宜家就要行動？ 行動嘅最佳時機喺尋日 之後就係今日 
 Why act now? The best time to act was yesterday. The second best time is today. 
https://www.ted.com/talks/brent_loken_can_we_create_the_perfect_farm/transcript?/transcript&language=zh
https://www.ted.com/talks/brent_loken_can_we_create_the_perfect_farm/transcript?/transcript&language=en
 喺大概一萬年前人類就開始耕種 呢次農業革命係我哋歷史轉折點 令到我哋可以定居、開始發展同創造 簡單啲講，農業就係文明基石 
 About 10,000 years ago, humans began to farm. This agricultural revolution was a turning point in our history that enabled people to settle, build and create. In short, agriculture enabled the existence of civilization. 
https://www.ted.com/talks/matt_walker_how_sleep_can_improve_your_immunity/transcript?/transcript&language=zh
https://www.ted.com/talks/matt_walker_how_sleep_can_improve_your_immunity/transcript?/transcri

In [ ]:
print('Number of Talks: ' + str(count))
print('Length of corpus: ' + str(corpus_length))
print('Time: ' + str(end - start))

Number of Talks: 130
Length of corpus: 2568
Time: 1393.9248530864716


In [ ]:
pieces=[]
if os.getcwd() != '/content/talk':
  os.chdir('talks')
for file in glob.glob('*.csv'):
    print(file)
    frame = pd.read_csv(os.getcwd() + '/' + file, sep='\t', encoding='utf-8')
    pieces.append(frame)
df = pd.concat(pieces, ignore_index=True)

stacey_kramer_the_best_gift_i_ever_survived.csv
fahad_al_attiya_a_country_with_no_water.csv
thomas_insel_toward_a_new_understanding_of_mental_illness.csv
marcin_jakubowski_open_sourced_blueprints_for_civilization.csv
lisa_bu_how_books_can_open_your_mind.csv
ron_mccallum_how_technology_allowed_me_to_read.csv
ron_finley_a_guerrilla_gardener_in_south_central_la.csv
a_j_jacobs_the_evolution_of_the_coffee_cup_lid.csv
tom_hulme_what_can_we_learn_from_shortcuts.csv
anika_paulson_how_i_found_myself_through_music.csv
peter_attia_is_the_obesity_crisis_hiding_a_bigger_problem.csv
peter_doolittle_how_your_working_memory_makes_sense_of_the_world.csv
ze_frank_are_you_human.csv
louie_schwartzberg_the_hidden_beauty_of_pollination.csv
murray_gell_mann_the_ancestor_of_language.csv
geoffrey_canada_our_failing_schools_enough_is_enough.csv
keller_rinaudo_a_mini_robot_powered_by_your_phone.csv
elizabeth_cox_what_is_a_coronavirus.csv
sarah_parcak_archaeology_from_space.csv
bill_gates_the_next_outbreak_we_re_

In [ ]:
os.chdir('/content/')
df.to_csv('All_TED_TALKS_DATA_1.csv',sep='\t',encoding='utf-8')

In [ ]:
df

,Unnamed: 0,zh,en
0,0,如果你可以的话，幻想一份礼物 请你在你的脑海里描绘 不是很大 高尔夫球大小 所以再想象它是...,"Imagine, if you will — a gift. I'd like for y..."
1,1,重新定义你对于 精神和信仰的认知 你对你的身体 会有新的认识和信任 你会有超凡的生命和精力...,It will redefine your sense of spirituality a...
2,2,你会因此改变 被启发 被鼓舞 感到自己的卑微 你的生命将会有新的意义 平静 健康 宁静 快...,"You'll be challenged, inspired, motivated and..."
3,3,我猜你们现在肯定很想知道 这究竟是什么 哪里可以买得到 亚马逊？ 是不是苹果公司出的？ 用...,By now I know you're dying to know what it is...
4,4,就算我现在已经康复了 我并不希望你会获得这份礼物 我不觉得你们想要它 但是我不能改变我自己...,"And while I'm okay now, I wouldn't wish this ..."
...,...,...,...
2563,8,（掌聲）,(Applause)
2564,9,系啦，你要將事實同偏見分開。 我嘅小兒書， 有好得意，毛茸茸嘅動物。 不過，佢地玩梗地緣政...,"You have to separate fact from bias, right. E..."
2565,10,我要宣稱，藝術同創新， 系培養同情，同感，同理入邊好重要嘅部份。 你明白，我唔可以保證我嘅...,I'm making an argument that art and creativit...
2566,11,非常感谢。,Thank you very much.
